# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-13 03:28:03] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-13 03:28:03] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-13 03:28:03] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-13 03:28:05] INFO trace.py:69: opentelemetry package is not installed, tracing disabled


[2025-11-13 03:28:05] WARNING memory_pool_host.py:36: Current platform not support pin_memory


...

Capturing batches (bs=104 avail_mem=48.90 GB):  15%|█▌        | 3/20 [00:00<00:02,  7.91it/s]

Capturing batches (bs=64 avail_mem=48.87 GB):  30%|███       | 6/20 [00:00<00:01,  9.53it/s]

Capturing batches (bs=40 avail_mem=48.86 GB):  55%|█████▌    | 11/20 [00:00<00:00, 13.95it/s]

Capturing batches (bs=16 avail_mem=48.84 GB):  65%|██████▌   | 13/20 [00:01<00:00, 14.74it/s]

Capturing batches (bs=1 avail_mem=48.82 GB): 100%|██████████| 20/20 [00:01<00:00, 12.63it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Amin Alqadr and I am a Computer Science student at the University of Essex. As such, I am a young man who is about to embark on a new chapter in my life and I am very excited and eager to start my career. I am also interested in the field of Artificial Intelligence and I believe it to be a very exciting area of study. 

I am currently enrolled in the Computer Science department at the University of Essex where I will be taking the University of Essex Computer Science 101: Introduction to Programming and Algorithms. I am passionate about programming and I feel that I can do great things if I learn and
Prompt: The president of the United States is
Generated text:  a very important person. The president takes care of the country and represents the country on the world stage. He is the head of the government and the commander in chief of the armed forces. The president has the ability to change the laws of the country. The president also has the p

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm a [job title] with [number of years] years of experience in [industry]. I'm a [job title] with [number of years] years of experience in [industry]. I'm a [job title] with [number of years] years of experience in [industry]. I'm a [job title] with [number of years] years of experience in [industry]. I'm a [job title] with [number of years] years of experience in [industry]. I'm a [job title] with [number of years

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. It is the largest city in France and the third largest in the world. It is known for its rich history, beautiful architecture, and vibrant culture. Paris is home to many famous landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. The city is also known for its annual festivals and cultural events. Paris is a popular tourist destination and a major economic center in France. It is a major hub for business, finance, and government. The city is also home to many important museums and cultural institutions. Paris is a city of contrasts, with its modern architecture and historical landmarks blending seamlessly. It

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing for more sophisticated and nuanced decision-making. This could lead to more personalized and context-aware AI systems that can better understand and respond to the needs of individuals.

2. Greater emphasis on ethical considerations: As AI becomes more integrated with human intelligence, there will be a greater emphasis on ethical considerations. This could lead to more robust and transparent AI systems that are designed to minimize bias and ensure fairness and justice.

3. Increased use of AI for creative and artistic endeavors: AI is already being used in creative



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I'm a [Role] at [Company/Agency]. I enjoy [Company/Agency's mission, value, or standout feature]. My background is [number of years working in the industry, and what skills or experiences I bring to the table]. I'm always up for [Any challenges or opportunities that might arise in the future]. Thanks for taking the time to meet me. How can I assist you today?
Hello! My name is [Name] and I'm a [Role] at [Company/Agency]. I'm excited to meet you today! How can I assist you today?
I look forward to meeting

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, which is renowned for its iconic landmarks such as the Eiffel Tower and the Notre-Dame Cathedral. The city is also famous for its vibrant cultural scene, rich history, and diverse neighborhoods. Other notable attractions include the Louvre Mus

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

]

...

 a

 greater

 emphasis

 on ethical

 considerations

In [6]:
llm.shutdown()